In [ ]:
%matplotlib notebook
#add comments
import os
import sys
import csv
import pickle
from collections import Counter
from datetime import datetime

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn
from torch.utils.data import DataLoader
from network import TweetyNet
import librosa
from librosa import display
from microfaune.audio import wav2spc, create_spec, load_wav
from glob import glob

from torch.utils.data import Dataset
from CustomAudioDataset import CustomAudioDataset

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import scipy
import IPython.display as ipd

from TweetyNetModel import TweetyNetModel  

from pydub import AudioSegment
import os

In [ ]:
import pydub
train = True
fineTuning = False
#needs at least 80 for mel spectrograms ## may be able to do a little less, but must be greater than 60
n_mels=72 # The closest we can get tmeporally is 72 with an output of 432 : i think it depends on whats good
#this number should be proportional to the length of the videos. 
num_workers=2

data = pd.read_csv("C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing/automated_cosmos_tweety_to_file.csv")
datasets_dir = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/Cosmos_data"
this_is_new_data = False
path_to_audio = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing"
data_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split"
data_path = "C:/Users/siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split"
new_folder = data_path
AudioSegment.converter = "C:/Users/siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"
pydub.AudioSegment.ffmpeg = "C:/Users/siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"


In [ ]:
main_data_location = ".\cosmos_data\Cosmos_data"

In [ ]:
import pandas as pd
import numpy as np


#TODO Add mutliple species support for ROC curves 
def annotation_chunker_no_duplicates(kaleidoscope_df, chunk_length, include_no_bird=False, bird=None):
    """
    Function that converts a Kaleidoscope-formatted Dataframe containing 
    annotations to uniform chunks of chunk_length. If there
    are mutliple bird species in the same clip, this function creates chunks
    for the more confident bird species.

    Note: if all or part of an annotation covers the last < chunk_length
    seconds of a clip it will be ignored. If two annotations overlap in 
    the same 3 second chunk, both are represented in that chunk
    Args:
        kaleidoscope_df (Dataframe)
            - Dataframe of annotations in kaleidoscope format

        chunk_length (int)
            - duration to set all annotation chunks
    Returns:
        Dataframe of labels with chunk_length duration 
        (elements in "OFFSET" are divisible by chunk_length).
    """

    #Init list of clips to cycle through and output dataframe
    #kaleidoscope_df["FILEPATH"] =  kaleidoscope_df["FOLDER"] + kaleidoscope_df["IN FILE"] 
    kaleidoscope_df['FILEPATH'] = kaleidoscope_df.loc[:,['FOLDER','IN FILE']].sum(axis=1)
    clips = kaleidoscope_df["FILEPATH"].unique()
    df_columns = {'FOLDER': 'str', 'IN FILE' :'str', 'CLIP LENGTH' : 'float64', 'CHANNEL' : 'int64', 'OFFSET' : 'float64',
                'DURATION' : 'float64', 'SAMPLE RATE' : 'int64','MANUAL ID' : 'str'}
    output_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in df_columns.items()})
    
    # going through each clip
    for clip in clips:
        clip_df = kaleidoscope_df[kaleidoscope_df["FILEPATH"] == clip]
        path = clip_df["FOLDER"].unique()[0]
        file = clip_df["IN FILE"].unique()[0]
        birds = clip_df["MANUAL ID"].unique()
        sr = clip_df["SAMPLE RATE"].unique()[0]
        clip_len = clip_df["CLIP LENGTH"].unique()[0]

        # quick data sanitization to remove very short clips
        # do not consider any chunk that is less than chunk_length
        if clip_len < chunk_length:
            continue
        potential_annotation_count = int(clip_len)//int(chunk_length)

        # going through each species that was ID'ed in the clip
        arr_len = int(clip_len*1000)
        species_df = clip_df#[clip_df["MANUAL ID"] == bird]
        human_arr = np.zeros((arr_len))
        # looping through each annotation
        #print("========================================")
        for annotation in species_df.index:
            #print(species_df["OFFSET"][annotation])
            minval = int(round(species_df["OFFSET"][annotation] * 1000, 0))
            # Determining the end of a human label
            maxval = int(
                round(
                    (species_df["OFFSET"][annotation] +
                        species_df["DURATION"][annotation]) *
                    1000,
                    0))
            # Placing the label relative to the clip
            human_arr[minval:maxval] = 1
        # performing the chunk isolation technique on the human array

        for index in range(potential_annotation_count):
            #print("=======================")
            #print("-----------------------------------------")
            #print(index)
            chunk_start = index * (chunk_length*1000)
            chunk_end = min((index+1)*chunk_length*1000,arr_len)
            chunk = human_arr[int(chunk_start):int(chunk_end)]
            if max(chunk) >= 0.5:
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #Handle birdnet output edge case
                #print("-------------------------------------------")
                #print(sum(clip_df["DURATION"] == 3))
                #print(sum(clip_df["DURATION"] == 3)/clip_df.shape[0])
                #print("-------------------------------------------")
                if(sum(clip_df["DURATION"] == 3)/clip_df.shape[0] == 1):
                    #print("Processing here duration")
                    overlap = (clip_df["OFFSET"]+0.5 >= (annotation_start)) & (clip_df["OFFSET"]-0.5 <= (annotation_start))
                    annotation_df = clip_df[overlap]
                    #print(annotation_start, np.array(clip_df["OFFSET"]), overlap)
                    #print(annotation_df)
                else:
                    #print("Processing here")
                    overlap = is_overlap(clip_df["OFFSET"], clip_df["OFFSET"] + clip_df["DURATION"], annotation_start, annotation_start + chunk_length)
                    #print(overlap)
                    annotation_df = clip_df[overlap]
                    #print(annotation_df)
                
                #updating the dictionary
                if ('CONFIDENCE' in clip_df.columns):
                    annotation_df = annotation_df.sort_values(by="CONFIDENCE", ascending=False)
                    row["CONFIDENCE"] = annotation_df.iloc[0]["CONFIDENCE"]
                else:
                    #The case of manual id, or there is an annotation with no known confidence
                    row["CONFIDENCE"] = 1
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = annotation_df.iloc[0]["MANUAL ID"] 
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
            elif(include_no_bird):
                #print(max(chunk))
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #updating the dictionary
                row["CONFIDENCE"] = 0
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = "no bird"
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
    
    return output_df


def is_overlap(offset_df, end_df, chunk_start, chunk_end):
    is_both_before = (chunk_end < offset_df) & (chunk_start < offset_df)
    is_both_after = (end_df < chunk_end) & (end_df < chunk_start)
    return (~is_both_before) & (~is_both_after)
    
    
    interval = pd.Interval(left=offset_df, right=end_df)
    print(interval)


def split_save_files(dataframe_of_data):
    chunked_df = annotation_chunker_no_duplicates(dataframe_of_data, 3, include_no_bird=True)
    
    
    chunked_df.apply()
    

In [ ]:


#
chunked_df = annotation_chunker_no_duplicates(data, 3, include_no_bird=True)
chunked_df

In [ ]:
chunked_df.iloc[0]["IN FILE"]

In [ ]:
import pydub


In [ ]:
from pydub import AudioSegment

import os

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path_to_audio) if isfile(join(path_to_audio, f))]
chunked_df = chunked_df[chunked_df["IN FILE"].isin(onlyfiles)]


chunked_renamed_df = chunked_df.copy()
count = 0
for filename in np.unique(chunked_df["IN FILE"]):
    file_df = chunked_df[chunked_df["IN FILE"] == filename]
    if (len(filename.split(".")) != 2):
        break
    file, file_type = filename.split(".")
    for index,row in file_df.iterrows():
        t1 = row["OFFSET"] * 1000
        # end time in milliseconds
        t2 = (row["OFFSET"] + 3) * 1000
        new_name = file + "_" + str(int(row["OFFSET"])) +"." + file_type
        new_path = new_folder + "/" + new_name
        
        if(this_is_new_data):
            newAudio = AudioSegment.from_mp3(path_to_audio + "/" + filename)
            newAudio = newAudio[t1:t2]
            newAudio.export(new_path, format="mp3")

        chunked_renamed_df.at[index, "IN FILE"] = new_name
        chunked_renamed_df.at[index, "FOLDER"] = new_folder
        count += 1
        print(count, "out of 6451")

        
chunked_renamed_df = chunked_renamed_df[chunked_renamed_df["IN FILE"] != "TO DELETE"]
chunked_renamed_df
    
    

In [ ]:
x = chunked_renamed_df.groupby(["IN FILE", "OFFSET"]).count().sort_values(by="MANUAL ID", ascending=False)
x.shape[0]

In [ ]:
def show_mel_spectrogram(spec, uid):
    fig, ax = plt.subplots()
    M_db_bird = librosa.power_to_db(spec, ref=np.max)
    img = librosa.display.specshow(M_db_bird, y_axis='mel', x_axis='time', ax=ax)
    ax.set(title=uid)
    plt.show()

In [ ]:
def load_dataset(data_path, use_dump=True, csv="labels.csv", kalediscope=False):
    mel_dump_file = os.path.join(data_path, "mel_dataset.pkl")
    if os.path.exists(mel_dump_file) and use_dump:
        with open(mel_dump_file, "rb") as f:
            dataset = pickle.load(f)
    
    elif kalediscope:
        dataset = compute_feature_kaledoscope_labels(data_path, csv)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    else:
        dataset = compute_feature(data_path, csv)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    inds = [i for i, x in enumerate(dataset["X"]) if x.shape[1] == 431]
    X = np.array([dataset["X"][i].transpose() for i in inds])
    Y = np.array([int(dataset["Y"][i]) for i in inds])
    uids = [dataset["uids"][i] for i in inds]
    return X, Y, uids

def compute_feature(data_path, csv = "labels.csv"):
    print(f"Compute features for dataset {os.path.basename(data_path)}")
    labels_file = os.path.join(data_path, csv)
    print(labels_file)
    if os.path.exists(labels_file):
        with open(labels_file, "r") as f:
            reader = csv.reader(f, delimiter=',')
            labels = {}
            next(reader)  # pass fields names
            for name, _, y in reader:
                labels[name] = y
    else:
        print("Warning: no label file detected.")
        wav_files = glob(os.path.join(data_path, "wav/*.wav"))
        labels = {os.path.basename(f)[:-4]: None for f in wav_files}
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        X.append(spc)
        Y.append(y)
        uids.append(file_id)
        i += 1
    return {"uids": uids, "X": X, "Y": Y}

def compute_feature_df(data_path, df):
    
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        X.append(spc)
        Y.append(y)
        uids.append(file_id)
        i += 1
    return {"uids": uids, "X": X, "Y": Y}

def split_dataset(X, Y, test_size=0.2, random_state=0):
    split_generator = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    ind_train, ind_test = next(split_generator.split(X, Y))
    X_train, X_test = X[ind_train, :, :], X[ind_test, :, :]
    Y_train, Y_test = Y[ind_train], Y[ind_test]
    return ind_train, ind_test

In [ ]:
species = list(np.unique(chunked_renamed_df["MANUAL ID"]))
chunked_renamed_df["y"] = chunked_renamed_df["MANUAL ID"].apply(lambda x: species.index(x))
chunked_renamed_df

In [ ]:
import os
from pydub import AudioSegment

# You dont need the number of files in the folder, just iterate over them directly using:
if (this_is_new_data):
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(folder + '/' + file)
        if ext == ".mp3":
            #os.remove(folder + '/' + file) 

            mp3_sound = AudioSegment.from_mp3(folder + '/' + file)
            mp3_sound.export("{0}.wav".format(name), format="wav")

In [ ]:
import os
from pydub import AudioSegment

if(this_is_new_data):
    # You dont need the number of files in the folder, just iterate over them directly using:
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".mp3":
            print(folder)
            try:
                os.remove(folder + "/" + file) 
            except Exception as e:
                print(e)
            

In [ ]:
chunked_renamed_df["IN FILE"][201]

In [ ]:
chunked_renamed_df["FOLDER"] = main_data_location

In [ ]:
chunked_renamed_df["IN FILE"] = chunked_renamed_df["IN FILE"].apply(lambda x: x.replace(".mp3", ".wav"))
chunked_renamed_df["IN FILE"][201]

In [ ]:
import os
from pydub import AudioSegment

# You dont need the number of files in the folder, just iterate over them directly using:
if(this_is_new_data):
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        sound = AudioSegment.from_wav(folder + "/" + file)
        sound = sound.set_channels(1)
        sound.export(folder + "/" + file, format="wav")


In [ ]:

#chunked_renamed_df = pd.read_csv("formated_tweetynet_ground_truth.csv")
i = 1
X = []
Y = []
uids = []
for index, row in chunked_renamed_df.iterrows():
    try:
        print(f"{i:04d}/{chunked_renamed_df.shape[0]}", end="\r")
        print(os.path.join(data_path, row["IN FILE"]))
        spc = wav2spc(data_path + '/' + row["IN FILE"], n_mels=n_mels)
        X.append(spc)
        Y.append(row["y"])
        uids.append(row["IN FILE"])
        i += 1
    except Exception as e:
        print(e)
        continue
dataset = {"uids": uids, "X": X, "Y": Y}

In [ ]:
dataset

In [ ]:


inds = [i for i, x in enumerate(dataset["X"]) if x.shape[1] == 130]
X = np.array([dataset["X"][i].transpose() for i in inds])
Y = np.array([int(dataset["Y"][i]) for i in inds])
uids = [dataset["uids"][i] for i in inds]
X, Y, uids


In [ ]:
dataset["X"][0].shape[1]


In [ ]:
dataset

In [ ]:
#X0, Y0, uids0 = load_dataset(datasets_dir, csv="automated_cosmos_tweety_to_file.csv")#os.path.join(datasets_dir, "ff1010bird_wav"))
#print(X0)
#uids0, X0, Y0 =  uids, X,Y#load_dataset(os.path.join(datasets_dir, "warblrb10k_public_wav"))

#print(X0.shape)
#print(Y0.shape)
#print(len(uids0))

#X = np.concatenate([X0]).astype(np.float32)/255
#Y = np.concatenate([Y0])
#uids = np.concatenate([uids0])
#print(X.shape, Y.shape, uids.shape)
#del X0, Y0, uids0, X1, Y1, uids1 

In [ ]:
Counter(Y)

Actual work

In [ ]:
print(type(uids))
uids = np.array(uids)

In [ ]:
ind_train_val, ind_test = split_dataset(X, Y)
ind_train, ind_val = split_dataset(X[ind_train_val, :, :, np.newaxis], Y[ind_train_val], test_size=0.1)
X_train, X_test, X_val = X[ind_train, :, :, np.newaxis], X[ind_test, :, :, np.newaxis], X[ind_val, :, :, np.newaxis]
Y_train, Y_test, Y_val = Y[ind_train], Y[ind_test], Y[ind_val]
uids_train, uids_test, uids_val = uids[ind_train], uids[ind_test], uids[ind_val]
#del X, Y

print("Training set: ", Counter(Y_train))
print("Test set: ", Counter(Y_test))
print("Validation set: ", Counter(Y_val))
#very imbalanced dataset

In [ ]:
len(Counter(Y_train))
X_train[0].shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
tweetynet = TweetyNetModel(len(Counter(Y_train)), (1, n_mels, 431), device, binary=True, workers=num_workers)

In [ ]:
train_dataset = CustomAudioDataset(X_train, Y_train, uids_train)
test_dataset = CustomAudioDataset(X_test, Y_test, uids_test)
val_dataset = CustomAudioDataset(X_val, Y_val, uids_val)

In [ ]:
#train_dataset = train_dataset.type(torch.LongTensor)
#test_dataset = test_dataset.type(torch.LongTensor)
#val_dataset = val_dataset.type(torch.LongTensor)

In [ ]:
train_dataset[:10]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload


train_dataset = CustomAudioDataset(X_train, Y_train, uids_train)
test_dataset = CustomAudioDataset(X_test, Y_test, uids_test)
val_dataset = CustomAudioDataset(X_val, Y_val, uids_val)

history, test_out, start_time, end_time = tweetynet.train_pipeline(train_dataset, val_dataset, test_dataset, 
                                                                   lr=.001, batch_size=128,epochs=10, save_me=True,
                                                                   fine_tuning=False, finetune_path=None)

In [ ]:
len(train_dataset)

In [ ]:
end_time - start_time

In [ ]:
test_out


In [ ]:
chunked_renamed_df.to_csv("formated_tweetynet_ground_truth.csv")

# Visualization

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:
test = output[:, 0, :].numpy()





for species_index in range(output.shape[1]):
    print(species_index)
    test = output[:, species_index, :].numpy()
    print(test.shape)
    count = 0
    scores = []
    
    d["uid"] =  temp_uids.flatten()
    print(d["uid"].shape[0])
    
    for file in test:
        for slice_ in file:
            scores.append(slice_)
            count += 1
    d[species[species_index]] = scores
    print(species[species_index], count, len( d[species[species_index]]))
    
    


for key in d:
    print(len(d[key]))

new_preds = pd.DataFrame(d)
new_preds

In [ ]:
predictions = pd.DataFrame()
pred_one_hot = pd.DataFrame(columns=np.append("UID", test_dataset.unique_labels()))
species = list(np.unique(chunked_renamed_df["MANUAL ID"]))
tweetynet.model.eval()


scores_df = pd.DataFrame()
pred_df = pd.DataFrame()
label_df = pd.DataFrame()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)
with torch.no_grad():
    for i, data in enumerate(test_loader):
        #ACTUALLY RUNNING MODEL
        #Run model over test data
        inputs, labels, uids = data
        inputs = inputs.reshape(inputs.shape[0], 1, inputs.shape[1], inputs.shape[2])
        inputs, labels = inputs.to(tweetynet.device), labels.to(tweetynet.device)

        output = tweetynet.model(inputs, inputs.shape[0], labels.shape[0])
        
        #Maniplute data shape
        temp_uids = []
        if tweetynet.binary:
            labels = labels.detach()
            labels = torch.tensor([[x] * output.shape[-1] for x in labels]).to(tweetynet.device)
            temp_uids = np.array([[x] * output.shape[-1] for x in uids])
        else:
            for u in uids:
                for j in range(output.shape[-1]):
                     temp_uids.append(str(j) + "_" + u)
            temp_uids = np.array(temp_uids)
        
        
        #PREPRARE DATAFRAME FOR ONE HOT ENCODING VISULIZAITONS
        #Get the predictions for each
        d = {}
        p = {}
        l = {}
        label_d = {}
        scores = []
        
        prediction = torch.argmax(output, dim=1)
        d["uid"] =  temp_uids.flatten()
        p["uid"] =  temp_uids.flatten()
        for species_index in range(output.shape[1]):
            #get scores from output
            scores = []
            preds = []
            labels_ = []
            test = output[:, species_index, :].numpy()
            for file in test:
                for slice_ in file:
                    scores.append(slice_)
                    preds.append(0)
                    labels_.append(0)
                    count += 1

            d[species[species_index]] = scores
            p[species[species_index]] = preds
            l[species[species_index]] = labels_

         #Label predictions in one hot encoding
        new_scores = pd.DataFrame(d)
        new_pred = pd.DataFrame(p)
        new_labels = pd.DataFrame(l)
       
        counter = 0
        for index, row in new_pred.iterrows():
            if (index % 144 == 0 and index != 0):
                counter += 1
            species_index = prediction[counter][index % 144]
            row[species[species_index]] = 1
            new_pred.loc[index] = row
         
        counter = 0
        for index, row in new_labels.iterrows():
            if (index % 144 == 0 and index != 0):
                counter += 1
            species_index = labels[counter][index % 144]
            row[species[species_index]] = 1
            new_labels.loc[index] = row
        
        scores_df = scores_df.append(new_scores)
        pred_df = pred_df.append(new_pred)
        label_df = label_df.append(new_labels)
        print("next", i, len(test_loader))

print('Finished Testing')
scores_df, pred_df, label_df

next 0 20
next 1 20
next 2 20
next 3 20
next 4 20
next 5 20
next 6 20
next 7 20
next 8 20
next 9 20
next 10 20
next 11 20
next 12 20
next 13 20
next 14 20
next 15 20
next 16 20
next 17 20


In [ ]:
label_df

In [ ]:
labels